<a href="https://colab.research.google.com/github/SJSY126/Quantum/blob/master/Qiskit_pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiskit.org/textbook/ch-quantum-hardware/calibrating-qubits-openpulse.html

In [7]:
!pip install -q qiskit

In [8]:
with open('/content/drive/My Drive/ibmq.txt', 'r') as f:
   ibmq_api_key = f.readline()

IBMQ.save_account(ibmq_api_key)

configrc.store_credentials:WARNING:2020-08-02 12:29:25,699: Credentials already present. Set overwrite=True to overwrite.


In [24]:
from qiskit.tools.jupyter import *
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

ibmqfactory.load_account:WARNING:2020-08-02 12:46:36,715: Credentials are already in use. The existing account in the session will be replaced.


open pulseが使えるバックエンドか確認する。
現在使えるバックエンドは`ibmq_armonk`のみ（1qubit）

In [25]:
backend_config = backend.configuration()
assert backend_config.open_pulse, "Backend doesn't support Pulse"

In [26]:
dt = backend_config.dt
print(f"Sampling time: {dt*1e9} ns")    # The configuration returns dt in seconds, so multiply by
                                        # 1e9 to get nanoseconds

Sampling time: 0.2222222222222222 ns


In [31]:
backend_defaults = backend.defaults()
print(backend_defaults)

<PulseDefaults(<InstructionScheduleMap(1Q instructions:
  q0: {'id', 'u2', 'u1', 'u3', 'measure', 'x'}
Multi qubit instructions:
)>Qubit Frequencies [GHz]
[4.974457727987402]
Measurement Frequencies [GHz]
[6.993427855] )>


In [32]:
import numpy as np

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# We will find the qubit frequency for the following qubit.
qubit = 0

# The sweep will be centered around the estimated qubit frequency.
center_frequency_Hz = backend_defaults.qubit_freq_est[qubit]        # The default frequency is given in Hz
                                                                    # warning: this will change in a future release
print(f"Qubit {qubit} has an estimated frequency of {center_frequency_Hz / GHz} GHz.")

# scale factor to remove factors of 10 from the data
scale_factor = 1e-14

# We will sweep 40 MHz around the estimated frequency
frequency_span_Hz = 40 * MHz
# in steps of 1 MHz.
frequency_step_Hz = 1 * MHz

# We will sweep 20 MHz above and 20 MHz below the estimated frequency
frequency_min = center_frequency_Hz - frequency_span_Hz / 2
frequency_max = center_frequency_Hz + frequency_span_Hz / 2
# Construct an np array of the frequencies for our experiment
frequencies_GHz = np.arange(frequency_min / GHz, 
                            frequency_max / GHz, 
                            frequency_step_Hz / GHz)

print(f"The sweep will go from {frequency_min / GHz} GHz to {frequency_max / GHz} GHz \
in steps of {frequency_step_Hz / MHz} MHz.")

Qubit 0 has an estimated frequency of 4.974457727987402 GHz.
The sweep will go from 4.954457727987402 GHz to 4.994457727987402 GHz in steps of 1.0 MHz.
